<a href="https://colab.research.google.com/github/Pailla34/Pailla34/blob/main/Stat_avanc%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas==2.0.3
import pandas as pd

In [2]:
from google.colab import files
import io
import pandas as pd

In [46]:
uploaded = files.upload()

Saving sondage-metiers-numeriques.csv to sondage-metiers-numeriques (7).csv


In [47]:
df = pd.read_csv(io.BytesIO(uploaded['sondage-metiers-numeriques (7).csv']))

In [48]:
df.head()

,checkbox-themes,radio-consulter-offres-publiques,radio-rejoindre,checkbox-pourquoi,checkbox-souhaits,radio-type-contrat,radio-lieu,text-lieu,number-salaire-min,number-salaire-souhait,checkbox-acceptation,text-condition,radio-genre,radio-metiers,text-autre-metier,number-experience,date
0,"Écologie, Justice, Affaires étrangères, Santé,...",Non,Oui,"Donner du sens à mon travail, Servir l'intérêt...","Au sein d'un ministère, Au sein d'une région, ...",CDD (2-3 ans),En région parisienne,Paris,3500,3900,"Travailler en méthodes agiles, Avoir un accès ...",NaN,Un homme,Développement fullstack,NaN,11.0,2019-07-22T13:04:38+00:00
1,"Écologie, Justice, Défense, Santé, Économie, É...",Non,Je ne sais pas,"Servir l'intérêt général, Travailler au côté d...","Au sein d'une région, Au sein d'un département...",CDI,En métropole,Nantes,3000,3500,Pouvoir travailler 1-2 jours par semaine en té...,NaN,Un homme,Développement frontend,NaN,15.0,2019-07-22T13:02:47+00:00
2,"Écologie, Santé, Éducation nationale, Agricult...",Non,Oui,"Donner du sens à mon travail, Servir l'intérêt...","Au sein d'une région, Au sein d'un département",CDD (2-3 ans),En métropole,Yonne,2000,3000,Pouvoir travailler 1-2 jours par semaine en té...,Droit à la déconnexion en dehors des heures de...,Un homme,Développement backend,NaN,1.0,2019-07-22T12:45:10+00:00
3,"Écologie, Éducation nationale, Culture",Non,Je ne sais pas,"Donner du sens à mon travail, Servir l'intérêt...","Au sein d'une région, Au sein d'un département...",CDI,En métropole,nantes,2000,2600,"Pouvoir travailler à 80-100% en télétravail, A...",Etre au 4/5 (si pas de téletravail),Une femme,Autre,QA / Tests / Automatisation,12.0,2019-07-22T11:39:42+00:00
4,"Éducation nationale, Enseignement supérieur et...",Oui,Oui,Pour avoir un travail stable,Au sein d'un ministère,CDI,En métropole,Avignon,1800,2500,"Avoir un accès à Internet sans aucun filtrage,...",NaN,Un homme,Administration système,NaN,17.0,2019-07-22T10:11:07+00:00


In [49]:
# Créer des variables dummies pour chaque colonne "checkbox"
df_themes_dummies = df['checkbox-themes'].str.get_dummies(sep=',')
df_pourquoi_dummies = df['checkbox-pourquoi'].str.get_dummies(sep=',')
df_souhaits_dummies = df['checkbox-souhaits'].str.get_dummies(sep=',')
df_acceptation_dummies = df['checkbox-acceptation'].str.get_dummies(sep=',')

# Ajouter les nouvelles colonnes au dataframe principal
df = pd.concat([df, df_themes_dummies, df_pourquoi_dummies, df_souhaits_dummies, df_acceptation_dummies], axis=1)



In [50]:
# Remplacer 'je ne sais pas' par 0
df['radio-rejoindre'] = df['radio-rejoindre'].map({'Oui': 1, 'Non': 0, 'Je ne sais pas': 0})

# Vérifier les résultats
print(df['radio-rejoindre'])


0      1
1      0
2      1
3      0
4      1
      ..
443    1
444    1
445    0
446    1
447    0
Name: radio-rejoindre, Length: 448, dtype: int64


In [51]:
# Ajouter le nombre d'éléments dans chaque colonne "checkbox"
df['N_themes'] = df['checkbox-themes'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)
df['N_pourquoi'] = df['checkbox-pourquoi'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)
df['N_souhaits'] = df['checkbox-souhaits'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)
df['N_acceptation'] = df['checkbox-acceptation'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)

# Vérifier les nouvelles colonnes
df[['N_themes', 'N_pourquoi', 'N_souhaits', 'N_acceptation']].head()


,N_themes,N_pourquoi,N_souhaits,N_acceptation
0,10,4,4,5
1,13,4,3,3
2,5,4,2,8
3,3,3,3,4
4,2,1,1,2


In [52]:
from sklearn.model_selection import train_test_split

# Variables explicatives et cible
features = ['number-salaire-souhait', 'N_acceptation', 'N_pourquoi', 'N_themes', 'N_souhaits', 'number-experience']
target = 'radio-rejoindre'  # Colonne représentant la volonté de rejoindre

X = df[features]
y = df[target]
print(df[target])
# Séparer les données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


0      1
1      0
2      1
3      0
4      1
      ..
443    1
444    1
445    0
446    1
447    0
Name: radio-rejoindre, Length: 448, dtype: int64


In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Entraîner et évaluer les modèles avec Logistic Regression
for C in range(1, 11):
    model = LogisticRegression(C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"Régression logistique (C={C}) : Accuracy = {acc}")


Régression logistique (C=1) : Accuracy = 0.5333333333333333
Régression logistique (C=2) : Accuracy = 0.5333333333333333
Régression logistique (C=3) : Accuracy = 0.5333333333333333
Régression logistique (C=4) : Accuracy = 0.5333333333333333
Régression logistique (C=5) : Accuracy = 0.5333333333333333
Régression logistique (C=6) : Accuracy = 0.5333333333333333
Régression logistique (C=7) : Accuracy = 0.5333333333333333
Régression logistique (C=8) : Accuracy = 0.5333333333333333
Régression logistique (C=9) : Accuracy = 0.5333333333333333
Régression logistique (C=10) : Accuracy = 0.5333333333333333


In [55]:
from sklearn.ensemble import RandomForestClassifier

# Entraîner et évaluer les modèles avec Forêt Aléatoire
for min_samples_leaf in range(1, 22):
    model = RandomForestClassifier(n_estimators=100, min_samples_leaf=min_samples_leaf, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"Random F (min_samples_leaf={min_samples_leaf}) : Accuracy = {acc}")

Random F (min_samples_leaf=1) : Accuracy = 0.5333333333333333
Random F (min_samples_leaf=2) : Accuracy = 0.4888888888888889
Random F (min_samples_leaf=3) : Accuracy = 0.4888888888888889
Random F (min_samples_leaf=4) : Accuracy = 0.5444444444444444
Random F (min_samples_leaf=5) : Accuracy = 0.5111111111111111
Random F (min_samples_leaf=6) : Accuracy = 0.5333333333333333
Random F (min_samples_leaf=7) : Accuracy = 0.5222222222222223
Random F (min_samples_leaf=8) : Accuracy = 0.5222222222222223
Random F (min_samples_leaf=9) : Accuracy = 0.5
Random F (min_samples_leaf=10) : Accuracy = 0.4888888888888889
Random F (min_samples_leaf=11) : Accuracy = 0.4888888888888889
Random F (min_samples_leaf=12) : Accuracy = 0.4777777777777778
Random F (min_samples_leaf=13) : Accuracy = 0.4666666666666667
Random F (min_samples_leaf=14) : Accuracy = 0.4666666666666667
Random F (min_samples_leaf=15) : Accuracy = 0.4666666666666667
Random F (min_samples_leaf=16) : Accuracy = 0.4777777777777778
Random F (min_sa